# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET
path = '/Users/jason/svn/springboard/data-wrangling/data_wrangling_xml'

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( path +'/data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
document = ET.parse( path + '/data/mondial_database.xml' )

1. 10 countries with lowest infant mortality rates
--------------------------------------------------

In [7]:
root = document.getroot()

In [8]:
#find name of infant mortality tag
for child in root[0]:
    if child.tag.find('infant') == 0:
        im_tag = child.tag
        
print im_tag

infant_mortality


In [9]:
#Create list strings with country name
#and list of floats with infant mortality numbers
country = []
im_stat = []
for child in root:
    if ET.iselement(child.find(im_tag)):
        country.append(child.find('name').text), im_stat.append(float(child.find(im_tag).text))

In [10]:
#Create Pandas Series from lists
import pandas as pd
country = pd.Series(country)
infant_mortality = pd.Series(im_stat)

In [11]:
#Create Dataframe from Series
mondial_im = pd.DataFrame()
mondial_im['country'] = country
mondial_im['infant_mortality'] = infant_mortality

In [12]:
#Sort dataframe by infant mortality, then print first 10
mondial_im.sort_values('infant_mortality').head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


2. 10 cities with largest population
---------------------------------------

In [13]:
city_pop = []
city_name = []
#i.   Find all cities in each country
#ii.  Create a list with all cities names
#iii. Create a list with all populations
for node in root:
    child = node.findall('city')
    for elem in child:
        if ET.iselement(elem):
            city = elem.find('name')
            population = elem.find('population')
            if ET.iselement(population):
                city_pop.append(int(population.text))
                city_name.append(city.text)

In [14]:
import pandas as pd

#Create Pandas series of city names, with population as index
city_pop_series = pd.Series(city_name, city_pop)
city_pop_series.head()

192000     Tirana
62000     Shkodër
60000      Durrës
56000       Vlorë
53000     Elbasan
dtype: object

In [15]:
#sort cities in descending order by population
city_pop_series.sort_index(ascending=False).head(10)

10229262              Seoul
7055071           Hong Kong
6053000          Al Qahirah
5876000             Bangkok
3924435         Ho Chi Minh
3813814               Busan
3722082          New Taipei
3056146               Hanoi
2917000     Al Iskandariyah
2626138              Taipei
dtype: object

3. Sum of top 10 ethnic groups by population\*
---------------------------------------------
(\*from our dataset)

In [115]:
#Create the following lists
#(total_population and ethnic_percentage added for sanity check of ethnic_population)
country = []
ethnic_group = []
ethnic_population = []
total_population = []
ethnic_percentage = []
for child in root:
    groups = child.findall('ethnicgroup')
    for elem in groups:
        if elem.attrib.keys()[0] == 'percentage':
            country.append(child.find('name').text)
            ethnic_group.append(elem.text)
            total_population.append(int(child.find('population').text))
            ethnic_percentage.append(float(elem.attrib['percentage']))
            ethnic_population.append(int((float(elem.attrib['percentage']) / 100) * float(child.find('population').text)))

In [119]:
#Create dataframe from lists
import pandas as pd
ethnic_populations = pd.DataFrame({'country' : country, 'ethnic_group' : ethnic_group, 'ethnic_population' : ethnic_population, 'ethnic_percentage' : ethnic_percentage, 'total_population' : total_population } )
ethnic_populations.head()

,country,ethnic_group,ethnic_percentage,ethnic_population,total_population
0,Albania,Albanian,95.0,1153764,1214489
1,Albania,Greek,3.0,36434,1214489
2,Greece,Greek,93.0,1020033,1096810
3,Macedonia,Macedonian,64.2,519200,808724
4,Macedonia,Albanian,25.2,203798,808724


In [122]:
#Sort dataframe in desceding order by size of the ethnic population
#and print first 10
ethnic_populations.sort_values('ethnic_population', ascending=False).head(10)

,country,ethnic_group,ethnic_percentage,ethnic_population,total_population
176,China,Han Chinese,91.50,497555113,543776080
221,India,Indo-Aryan,72.00,171645355,238396327
345,United States,European,79.96,126187306,157813040
93,Russia,Russian,79.80,82033328,102798657
299,Japan,Japanese,99.40,81706273,82199470
39,Germany,German,91.50,62431178,68230796
220,India,Dravidian,25.00,59599081,238396327
165,United Kingdom,English,83.60,42314986,50616012
520,Nigeria,African,99.00,37481146,37859744
278,Indonesia,Javanese,45.00,32666486,72592192
